In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
params = {'text.latex.preamble' : [r'\usepackage{siunitx}', r'\usepackage{amsmath}']}
plt.rcParams.update(params)

/Users/jacob/opt/anaconda3/lib/python3.8/_collections_abc.py:832: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [283]:
# df_investors = pd.read_excel("cleaned_investors_final.xlsx") --> 25 Nov updated final investor list given tweet reduction
df_investors = pd.read_csv("cleaned_investors_final.csv")
df_investors.drop(columns='Unnamed: 0', inplace=True)

#data format for df_investors should be: partner_uuid, name, username, country_code, city

In [3]:
#load in the relevant tables
#investors = pd.read_excel("final_investor_list.xlsx")
investment_partners = pd.read_csv("investment_partners.csv")
funding_rounds = pd.read_csv("funding_rounds.csv")
ipos = pd.read_csv("ipos.csv")
acquisitions = pd.read_csv("acquisitions.csv")
organizations = pd.read_csv("organizations.csv")
investments = pd.read_csv('investments.csv')

In [285]:
#rename the column to match on and make final investors table, matching on investors' twitter handle
df_investors.rename(columns = {'twitter_handle':'un'}, inplace = True)
#investors = investors.merge(df_investors, how='left', on = 'un')

In [286]:
#create model_df by matching the investors to investment_partners table and dropping unnecessary investment_partners columns
df_investors.rename(columns = {'uuid': 'partner_uuid'}, inplace=True)
model_df = df_investors.merge(investment_partners, how='left', on='partner_uuid')
drop_cols = ['name_x', 'name_y', 'type', 'permalink', 'cb_url', 'rank', 'updated_at', 
             'created_at','partner_name', 'funding_round_name']
model_df.drop(columns=drop_cols, inplace=True)
model_df.rename(columns={'investor_uuid': 'investor_org_uuid', 'partner_uuid': 'investor_uuid', 'investor_name': 'investor_org_name'},inplace=True)
len(model_df)

9638

In [287]:
#rename funding_round uuid and isolate only the funding rounds that were announced before 2016
funding_rounds.rename(columns={'uuid':'funding_round_uuid'},inplace=True)
#funding_rounds_2016 = funding_rounds[funding_rounds['announced_on']<"2015-01-01"]
funding_rounds_2016 = funding_rounds

#link the org_uuid from the funding rounds to the model_df
model_df['org_uuid'] = model_df.funding_round_uuid.map(funding_rounds_2016.set_index('funding_round_uuid')['org_uuid'].to_dict())
model_df['investor_org_type'] = model_df.investor_org_uuid.map(organizations.set_index('uuid')['type'].to_dict())
model_df['investor_org_primary_role'] = model_df.investor_org_uuid.map(organizations.set_index('uuid')['primary_role'].to_dict())

print(len(model_df))

model_df['founded_on'] = model_df.org_uuid.map(organizations.set_index('uuid')['founded_on'].to_dict())
model_df = model_df[model_df['founded_on']<"2015-01-01"]
model_df['founded_on'] = pd.to_datetime(model_df['founded_on'], errors='coerce')
model_df['founded_on'] = model_df['founded_on'].dt.year.astype(int)


#drop the entries that did not have an org match and reset the index
model_df = model_df[model_df['org_uuid'].notna()]
model_df.reset_index(drop=True, inplace=True)
print(len(model_df))

9638
5260


In [288]:
#create a table summarizing the total amount raise for each funding round before 2016
org_amount_raised_2016 = pd.pivot_table(funding_rounds_2016, values='raised_amount_usd', index=['org_uuid'], aggfunc=np.sum)
org_amount_raised_2016['org_uuid'] = org_amount_raised_2016.index
org_amount_raised_2016.reset_index(drop=True, inplace=True)

#create a raise_amount_usd column that represents the total amount raised over all funding round for each organization
#model_df['raised_amount_usd'] = model_df.org_uuid.map(org_amount_raised_2016.set_index('org_uuid')['raised_amount_usd'].to_dict())
model_df['raised_amount_usd'] = model_df.org_uuid.map(organizations.set_index('uuid')['total_funding_usd'].to_dict())

model_df['raised_amount_usd'].fillna(0, inplace=True)
len(model_df)

5260

In [289]:
#create an IPO variable that returns 1 if the organization had an IPO and 0 if no IPO was reported
model_df['ipo'] = model_df.org_uuid.map(ipos.set_index('org_uuid')['went_public_on'].to_dict())
model_df['ipo'] = model_df['ipo'].isna()

for x in range(0, len(model_df['ipo'])):
    if model_df['ipo'].loc[x] == True:
        model_df['ipo'].loc[x] = 0
    else:
        model_df['ipo'].loc[x] = 1
len(model_df)

5260

In [290]:
#create an acquisition price variable that reports the price that an organization was acquired for
model_df['acq_price_usd'] = model_df.org_uuid.map(acquisitions.set_index('acquiree_uuid')['price_usd'].to_dict())
model_df['acq_price_usd'] = model_df['acq_price_usd'].fillna(0)
len(model_df)

5260

In [291]:
#create a suc_acq variable that reports whether an investment was a successful acquisition based on if the acquisition price was greater than 1.5x the total amount raised
model_df['suc_acq'] = model_df.index
for x in range(0, len(model_df)):
    if model_df['acq_price_usd'][x] >= 4*model_df['raised_amount_usd'][x] and model_df['acq_price_usd'][x] !=0 and model_df['raised_amount_usd'][x] != 0:
        model_df['suc_acq'][x]=1
    else:
        model_df['suc_acq'][x]=0
len(model_df)

5260

In [292]:
# create a successful exits dummy variable that is 1 if the investment IPO'd or was successfully acquired and 0 otherwise
model_df['suc_exit'] = model_df.index
for x in range(0, len(model_df)):
    if model_df['ipo'][x]==1 or model_df['suc_acq'][x]==1:
        model_df['suc_exit'][x]=1
    else:
        model_df['suc_exit'][x]=0
len(model_df)

#robustness iteration 1 --> only IPO
# model_df['suc_exit'] = model_df.index
# for x in range(0, len(model_df)):
#     if model_df['ipo'][x]==1:
#         model_df['suc_exit'][x]=1
#     else:
#         model_df['suc_exit'][x]=0
# len(model_df)

#robustness iteration 2 --> 4x acquisition
# model_df['suc_exit'] = model_df.index
# for x in range(0, len(model_df)):
#     if model_df['ipo'][x]==1 or model_df['suc_acq'][x]==1:
#         model_df['suc_exit'][x]=1
#     else:
#         model_df['suc_exit'][x]=0
# len(model_df)

5260

In [293]:
#create a column that concatenates the investor and organizations together to remove cases where an investors invests in the same venture through multiple investment rounds
model_df["investor_org_uuid_unique"] = model_df["investor_uuid"] + model_df["org_uuid"]
model_df = model_df.drop_duplicates(subset=["investor_org_uuid_unique"])
model_df.reset_index(drop=True, inplace=True)
len(model_df)

3475

In [294]:
#drop columns used to build suc_exit variable so that they do not bias models
drop_cols = ['investor_org_uuid_unique','ipo','acq_price_usd','suc_acq',]
model_df.drop(columns=drop_cols, inplace=True)

In [295]:
#add additional org columns for investments
model_df['org_category_groups_list'] = model_df.org_uuid.map(organizations.set_index('uuid')['category_groups_list'].to_dict())
model_df = model_df.dropna(subset=['org_category_groups_list'])
model_df.reset_index(drop=True, inplace=True)
model_df['org_category_groups_list'] = model_df['org_category_groups_list'].str.split(',')

#add organization country code
model_df['org_country_code'] = model_df.org_uuid.map(organizations.set_index('uuid')['country_code'].to_dict())

#add organization city
model_df['org_city'] = model_df.org_uuid.map(organizations.set_index('uuid')['city'].to_dict())

#add number of funding rounds
model_df['org_num_funding_rounds'] = model_df.org_uuid.map(organizations.set_index('uuid')['num_funding_rounds'].to_dict())

#add employee count
model_df['org_employee_count'] = model_df.org_uuid.map(organizations.set_index('uuid')['employee_count'].to_dict())

In [296]:
#create dummy columns for categoy list 
t0 = time.time()

unique_cats = model_df['org_category_groups_list'].explode().unique() 
for cat in unique_cats:
    model_df['org_cat_'+str(cat)] = ""
for x in range(0, len(model_df)):
    for cat in unique_cats:
        if cat in model_df['org_category_groups_list'][x]:
            model_df['org_cat_'+str(cat)][x]=1
        else:
            model_df['org_cat_'+str(cat)][x]=0
    #print(x)
t1 = time.time()
total = (t1 - t0)
print('cell took', total, 'seconds')
model_df.columns = model_df.columns.str.lower()
model_df.drop(columns=(['org_category_groups_list']),inplace=True)

cell took 16.831119060516357 seconds


In [297]:
#successful exit rate
print((model_df.suc_exit.sum() / len(model_df))*100, "percent successful exits in", len(model_df), "deals", model_df.investor_uuid.nunique(), "unique investors")

9.576002307470436 percent successful exits in 3467 deals 951 unique investors


In [298]:
robust_2 = model_df

In [301]:
robust_2 = robust_2[model_df.columns.tolist()]
robust_2.to_csv('model_df_rob2.csv')

In [302]:
print('test before:', model_bf.suc_exit.sum())
model_bf['suc_exit'] = robust_2['suc_exit']
print('test after:', model_bf.suc_exit.sum())

print('test before:', model_bf_facets.suc_exit.sum())
model_bf_facets['suc_exit'] = robust_2['suc_exit']
print('test after:', model_bf_facets.suc_exit.sum())

model_bf.to_csv('model_bf_rob2.csv')
model_bf_facets.to_csv('model_bf_facets_rob2.csv')

test before: 407
test after: 332
test before: 407
test after: 332


In [303]:
print('test before:', model_hc.suc_exit.sum())
model_hc['suc_exit'] = model_hc.investor_uuid.map(robust_2.set_index('investor_uuid')['suc_exit'].to_dict())
print('test after:', model_hc.suc_exit.sum())

print('test before:', model_bf_hc.suc_exit.sum())
model_bf_hc['suc_exit'] = model_bf_hc.investor_uuid.map(robust_2.set_index('investor_uuid')['suc_exit'].to_dict())
print('test after:', model_bf_hc.suc_exit.sum())

print('test before:', model_bf_facets_hc.suc_exit.sum())
model_bf_facets_hc['suc_exit'] = model_bf_facets_hc.investor_uuid.map(robust_2.set_index('investor_uuid')['suc_exit'].to_dict())
print('test after:', model_bf_facets_hc.suc_exit.sum())

test before: 340
test after: 205
test before: 340.0
test after: 205
test before: 340.0
test after: 205


In [304]:
model_hc.to_csv('model_hc_rob2.csv')
model_bf_hc.to_csv('model_bf_hc_rob2.csv')
model_bf_facets_hc.to_csv('model_bf_facets_hc_rob2.csv')

In [2862]:
# model_df.to_csv('model_df_2_dec.csv')
# model_df.to_csv('model_df_2_dec.csv')